In [ ]:
import datetime
import time
import netmiko
import paramiko
import pymysql
import os

acase_list = ["router","switch","server"]
scase_list = []
# cmd path list = cmd 파일의 경로값이 저장되어 있는 dict
ahost_list = [] # all host list =  특정 파일에서 읽어온 모든 host가 담긴 리스트
shost_list= [] # selected host list = 사용자가 선택한 host들이 담긴 list
cmd_list = [] # all cmd list = 사용자가 고른 파일 유형에 맞는 모든 명령어 목록을 담을 함수
scnt = 0 #스위치 반복시 증가하는 값

# db 질의 def query(database,cmd) return rows
def query(cmd):
    ip = "172.16.0.51"
    port = "3306"
    id = "network"
    pw = "asd123!@"
    db = pymysql.connect(host=ip, user=id, passwd=pw, charset='utf8', db="guideline")
    #db 객체를 호출하는 cursor 생성
    cur = db.cursor()
    cur.execute(cmd)
    rows = []
    if "SELECT" in cmd or "SHOW" in cmd:
        rows = cur.fetchall()   # DBMS에서 전달하는 응답값을 가져온다.
    elif "INSERT" in cmd:
        db.commit()
        rows = ["query ok"]
    return list(rows)

#1. 파일 선택 def selcase(fpath_list) return x
def selcase(acase_list, scase_list) :

    print("전체 케이스 목록입니다.")
    cnt = 1
    for i in acase_list :
        print("%d. %s"%(cnt,i),end="  ")
        cnt += 1
    txt = input("선택하려는 케이스의 번호를 ,를 붙여서 입력해주세요.").split(",")
    
    for k in txt :
        if k.isdigit() :
        	index = int(k)-1
        	scase_list.append(acase_list[index])
        else :
            print("숫자를 입력해주세요")

#2. 호스트 정보 읽고 저장 def gethost
def gethost(case) :
    cmd = "select * from host"
    rows = query(case,cmd)
    for i in rows:
        print(i)
        host = {}
        host["hostname"] = i[0]
        host["ip"] = i[1]
        host["username"] = i[2]
        host["password"] = i[3]
        if len(i) == 5 :
            host["os"] = i[4]
        ahost_list.append(host.copy())

#3. 전체 호스트 보여주고 선택한 호스트 저장 def selhost(ahost_list, shost_list) return x
def selhost(case,ahost_list, shost_list) :
    print("전체 호스트 목록입니다.")
    cnt = 1
    for i in ahost_list :
        print("%d. %s"%(cnt,i["hostname"]),end="  ")
        cnt += 1
    txt = input("선택하려는 호스트의 번호를 ,를 붙여서 입력해주세요.").split(",")
    #txt = ""
    #if case == "router" :
    #    txt = ["4","3","1","2","5","6","7","8","9"]
    #elif case == "switch" :
    #    txt = ["1","2","3","1","2","3"]
    for k in txt :
        if k.isdigit() :
            index = int(k)-1
            shost_list.append(ahost_list[index])
        else :
            print("숫자를 입력해주세요")

#4. 명령어 정보 읽고 저장 def getcmd(cpath) return x
def getcmd(case) :
    if case in ("router","switch"):
        cmd = "show tables like '%_%'"
        rows = query(case,cmd)
        rows = list(rows[1:])
        result = []
        for a in rows:
            a = str(a)
            a = a[2:-3]
            result.append(a)
    elif case == "server":
        result = ["key_exchange", "smb_server", "smb_client", "dns_server", "ftp_login", "ftp_anonymous", "install_php",
                 "nginx", "nfs_server", "nfs_client", "install_mariadb", "pydio", "pydio_db", "joomla", "joomla_db", "wp_setup", "wp", "wp_db",
                 "monitorix_server", "monitorix_client", "mrtg"]
    return result

#5. 전체 명령어 보여주고 선택한 명령어 저장 def selcmd(shost_list, cmd_list) return x
def selcmd(case,host, cmd_list) :
    
    print("현재 호스트는 %s 입니다."%(host["hostname"]))
    print("전체 명령어는 다음과 같습니다.")
    cnt = 1
    host["cmd"] = []
    txt = ""    
    for j in cmd_list:
    	print("%d. %s"%(cnt,j))
    	cnt += 1
            
    txt = input("선택하려는 명령어의 번호를 ,를 붙여서 입력해주세요.").split(",")
    if case == "router" :
        txt = "1,7,3,6,8,13,5,15,16,4,9,10,11,12,14,2"
        txt = txt.split(",")
        
    for k in txt :
        if k.isdigit() :
            index = int(k) - 1
            host["cmd"].append(cmd_list[index])


#6. 명령어 전달 및 결과값 저장 def para_connect(user) return result
def para_connect(cmd,time1) :
    print("connect 시작")
    ip = "172.16.0.51"
    user = "root"
    pwd = "asd123!@"
    cli = paramiko.SSHClient() # ssh 클라이언트 인스턴스를 생성 ---> cli 객체
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy)  # 접속할 때 에러 메시지 처리 
    cli.connect(ip,username=user,password=pwd) #ip,username,password로 ssh 연결
    connection = cli.invoke_shell()

    idle_time = 120
    time.sleep(1)  # 초기 버퍼 비우기
    if connection.recv_ready():
        connection.recv(65535)

    cmd_list = cmd.strip().split("\n")
    result_all = ""

    for cmd_item in cmd_list:
        print(f"▶ {cmd_item} 실행 중...")
        connection.send(cmd_item + "\n")
        result = ""
        last_data_time = time.time()

        while True:
            time.sleep(0.2)
            if connection.recv_ready():
                chunk = connection.recv(65535).decode('utf-8')
                result += chunk
                last_data_time = time.time()
            elif time.time() - last_data_time > idle_time:
                # idle_time 동안 데이터가 안 들어오면 종료된 것으로 판단
                break

        print(f"⏹ {cmd_item} 완료")
        result_all += f"\n\n==== {cmd_item} 결과 ====\n{result}"

    cli.close()
    
    return result_all # 디코드된 결과값 리턴

#6. 명령어 전달 및 결과값 저장 def net_connect(user) return result
def net_connect (host, cmd,time1):
    net_connect = netmiko.ConnectHandler(device_type="cisco_ios", ip=host["ip"],username=host["username"],password=host["password"],timeout=15) # ssh 연결
    net_connect.enable() #관리자 모드 실행
    cmdt = cmd.strip()
    #print(" %s를 시작합니다."%(cmdt)) # 명령어 시작 알림
    tmp = cmd.split("\n")
    if len(tmp) == 1 :
        result =net_connect.send_command(tmp[0])
    else :
        result = net_connect.send_config_set(tmp)
    time.sleep(time1)
    #print(result)
    #print(" %s가 완료되었습니다."%(cmdt)) # 명령어 완료 알림
    net_connect.disconnect()

    return result

def U_01 (host):
    is_safe = False
    score = "0"
    print("start contents : U-01")
    
    command = "grep -i '^PermitRootLogin' /etc/ssh/sshd_config"
    #result = para_connect(command, 5)
    result = "PermitRootLogin no"

    permit_line = ""

    for line in result.splitlines():
        if "PermitRootLogin" in line:
            permit_line = line.strip()
            break

    if "no" in permit_line.lower():
        is_safe = True
        score = "3"

    date = datetime.datetime.now().strftime("%Y-%m-%d")
    
    def escape(s):
        return s.replace("\\", "\\\\").replace("'", "\\'").replace('"', '\\"')

    result_escaped = escape(result)
    command_escaped = escape(command)

    is_safe_sql = 'NULL' if is_safe is None else int(is_safe)

    cmd = """
        INSERT INTO info (id, date, content, command, result, is_safe, score)
        VALUE (%s, '%s', '%s', '%s', '%s', %s, %s)
    """ % (
        host['id'], date, 'U-01', command_escaped, result_escaped, is_safe_sql, score
    )

    query(cmd)

    info_obj = info(
        id = host['id']
        date = date
        content = 'U-01'
        command = command
        result = result
        is_safe = is_safe
        score = score
    )

    return info_obj

def U_13 (host):
    is_safe = False
    score = "0"
    print("start contents : U-13")

    

host = {"id" : 1}

U_01(host)